# Item-Based Collaborative Filtering

We'll be building a full blown recommender system that looks at the movie ratings every user gave in order to recommend them movies they are most likely to love.

In [1]:
import pandas as pd

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('/Users/jacquesthibodeau/Python Data Science/ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3))

m_cols = ['movie_id', 'title']
movies = pd.read_csv('/Users/jacquesthibodeau/Python Data Science/ml-100k/u.item', encoding = "ISO-8859-1", sep='|', names=m_cols, usecols=range(2))

ratings = pd.merge(movies, ratings)

ratings.head()

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3
